In [ ]:
#   ▄████████    ▄████████ ████████▄  ████████▄   ▄█      ███             ▄████████    ▄████████  ▄████████    ▄█    █▄     ▄█   ▄█    █▄     ▄████████    ▄████████
#   ███    ███   ███    ███ ███   ▀███ ███   ▀███ ███  ▀█████████▄        ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ███    █▀  ███    ███ ███    ███ ███▌    ▀███▀▀██        ███    ███   ███    ███ ███    █▀    ███    ███   ███▌ ███    ███   ███    █▀    ███    ███
#  ▄███▄▄▄▄██▀  ▄███▄▄▄     ███    ███ ███    ███ ███▌     ███   ▀        ███    ███  ▄███▄▄▄▄██▀ ███         ▄███▄▄▄▄███▄▄ ███▌ ███    ███  ▄███▄▄▄      ▄███▄▄▄▄██▀
# ▀▀███▀▀▀▀▀   ▀▀███▀▀▀     ███    ███ ███    ███ ███▌     ███          ▀███████████ ▀▀███▀▀▀▀▀   ███        ▀▀███▀▀▀▀███▀  ███▌ ███    ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀
# ▀███████████   ███    █▄  ███    ███ ███    ███ ███      ███            ███    ███ ▀███████████ ███    █▄    ███    ███   ███  ███    ███   ███    █▄  ▀███████████
#   ███    ███   ███    ███ ███   ▄███ ███   ▄███ ███      ███            ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ██████████ ████████▀  ████████▀  █▀      ▄████▀          ███    █▀    ███    ███ ████████▀    ███    █▀    █▀    ▀██████▀    ██████████   ███    ███
#   ███    ███                                                                         ███    ███                                                          ███    ███

# Reddit Archiver - An easy way to archive and view your Reddit Account. 

Downloads your profile info and posts/comments from your input reddit account. Creates a presentable webpage of your arhcived profile that can be saved locally. 

* Allows user to input desired Reddit Username and output directory for archive file via command line
* Downloads user data using BeautifulSoup.
* Saves data to JSON file
* Loads JSON File into HTML webpage and saves webpage to desired directory


By [dandex200](https://github.com/dandex200/) on Github


#### Imports

In [20]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tkinter import filedialog, Tk

## User Input

In [27]:
#Username Input
user_input = input("Enter desired Reddit Username to Archive:")

#Output Directory Input using Tkinter
confirm_input = input("Choose Archive File Output Directory? (Y/N)")

if confirm_input.lower() == 'y':
    root = Tk() 
    root.withdraw()
    output_dir = filedialog.askdirectory()
    root.destroy()

else:
    print("No output directory selected - will save to current directory.")
    

## Scraping

In [14]:
#Test BOX

url = 'https://old.reddit.com/user/NPU-F' 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')

# extract the username from the page title
username = soup.title.text.split(' ')[2]

# extract the comment karma from the user info box
comment_karma_box = soup.find('span', {'class': 'comment-karma'})
comment_karma = comment_karma_box.text if comment_karma_box is not None else ''

# extract the post karma from the user info box
post_karma_box = soup.find('span', {'class': 'karma'})
post_karma = post_karma_box.text if post_karma_box is not None else ''

# create a dictionary with the scraped data
data = {'username': username, 'comment_karma': comment_karma, 'post_karma': post_karma}

# output the dictionary to JSON format
json_data = json.dumps(data)

# print the JSON data to the console
print(json_data)


{"username": "NPU-F", "comment_karma": "14,739", "post_karma": "27,087"}
